### WebScrapping

In [7]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
# Load the CSV file
csv_file_path = 'Fruit_Site_Links  - Sheet1.csv'
data = pd.read_csv(csv_file_path)

# Display the first few rows to confirm the data structure
data.head(10)


,Fruits,Links
0,Banana1,https://www.google.com/search?sca_esv=08590015...
1,Mango1,https://www.google.com/search?sca_esv=08590015...
2,Orange1,https://www.google.com/search?sca_esv=08590015...
3,Orange2,https://www.google.com/search?q=isolated+orang...
4,Banana2,https://www.google.com/search?q=banana+on+whit...
5,Banana3,https://www.google.com/search?sca_esv=08590015...
6,Mango2,https://www.gettyimages.com/search/2/image?phr...
7,Orange3,https://www.google.com/search?sca_esv=08590015...
8,apple3,https://www.stickpng.com/img/food/fruits/apple...
9,apple1,https://www.istockphoto.com/search/2/image-fil...


In [32]:
# Function to download an image from a URL and save it locally
def download_image(url, folder_name, image_name):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        image_path = os.path.join(folder_name, image_name)
        with open(image_path, 'wb') as out_file:
            out_file.write(response.content)
    except Exception as e:
        print(f"Failed to download {url}: {e}")


In [34]:
def scrape_images(url, folder_name):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    session = requests.Session()
    try:
        response = session.get(url, headers=headers)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Find all image tags
        image_tags = soup.find_all('img')
        
        # Download each image
        for i, img_tag in enumerate(image_tags):
            if i >= 50:  # Limit to 50 images
                break
            img_url = img_tag.get('src')
            if img_url and not img_url.startswith('http'):
                img_url = url + img_url  # Handle relative URLs
            if img_url:
                # Extract the base name of the folder
                base_name = os.path.basename(folder_name)
                # Construct the image name
                image_name = f"{base_name}_{i + 1}.jpg"
                download_image(img_url, folder_name, image_name)
    except Exception as e:
        print(f"Failed to scrape {url}: {e}")


In [35]:


# Create a folder to save the images
os.makedirs('fruit_images', exist_ok=True)


In [36]:
# Iterate over the links in the CSV file and scrape images from each website
for index, row in data.iterrows():
    fruit = row['Fruits']
    link = row['Links']
    folder_name = os.path.join('fruit_images', fruit)
    os.makedirs(folder_name, exist_ok=True)
    scrape_images(link, folder_name)

print("Image scraping and downloading completed.")

Failed to download https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQ-MMpbSO9RTjiAqmgYNkpwPhE42HIaicdoshgyey7OrA2Sj0RHgSU8GKqrQUU&s: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Failed to scrape https://www.gettyimages.com/search/2/image?phrase=single%20mango%20isolated&sort=mostpopular&license=rf%2Crm: 403 Client Error: Forbidden for url: https://www.gettyimages.com/search/2/image?phrase=single%20mango%20isolated&sort=mostpopular&license=rf%2Crm
Failed to scrape https://www.istockphoto.com/search/2/image-film?istockcollection=signature%2Csignatureplus&page=3&phrase=apple%20fruit: 403 Client Error: Forbidden for url: https://www.istockphoto.com/search/2/image-film?istockcollection=signature%2Csignatureplus&page=3&phrase=apple%20fruit
Image scraping and downloading completed.
